## Import Libraries

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchaudio
from tqdm import tqdm

In [2]:
# HyperParameters
dataset_path = "/content/drive/MyDrive/Dataset/Audio_Dataset_Chunk"
batch_size = 32
learning_rate = 0.001
epochs = 60

## Model

In [3]:
# Model
class Model(nn.Module):
    def __init__(self, n_input=1, n_output=8, stride=4, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.softmax(x, dim=1)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)

## Load Dataset

In [5]:
# DATASET
class AudioDataset(Dataset):
  def __init__(self,root):
    self.dir_path = root
    self.classes = os.listdir(self.dir_path)
    
    self.data_paths=[]
    self.labels=[]
    for names in os.listdir(self.dir_path):
      folder = os.listdir(os.path.join(self.dir_path,names))
      for file in folder:
        self.data_paths.append(os.path.join(self.dir_path,names,file))
        self.labels.append(self.classes.index(names))   

  def __len__(self):
    return len(self.labels)

  def __getitem__(self,index):
    data_path = self.data_paths[index]
    label = self.labels[index]
    signal, sample_rate = torchaudio.load(data_path)
    signal = torch.mean(signal, dim=0, keepdim=True)
    new_sample_rate = 8000
    transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
    signal = transform(signal)

    return signal, label

In [6]:
dataset = AudioDataset(dataset_path)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
def accuracy(preds,labels):
    _,pred_max = torch.max(preds,1)
    acc = torch.sum(pred_max==labels) / len(preds)
    return acc

In [8]:
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

## Training

In [9]:
# TRAIN
for epoch in range(epochs):
  train_acc = 0.0
  train_loss = 0.0
  model.train()
  for audios, labels in tqdm(train):
    labels = labels.to(device)
    audios = audios.to(device)
    labels_one_hot = F.one_hot(labels,num_classes=8).type(torch.FloatTensor).to(device)
    optimizer.zero_grad()
    pred = model(audios)
    loss = loss_function(pred,labels_one_hot)
    loss.backward()
    optimizer.step()

    train_loss += loss
    train_acc += accuracy(pred,labels)

  total_train_loss = train_loss / len(train)
  total_train_acc = train_acc / len(train)
  print(f"Epochs: {epoch+1}, Accuracy: {total_train_acc}, Loss: {total_train_loss}")

100%|██████████| 22/22 [03:13<00:00,  8.80s/it]


Epochs: 1, Accuracy: 0.4488636255264282, Loss: 1.9425605535507202


100%|██████████| 22/22 [00:04<00:00,  4.40it/s]


Epochs: 2, Accuracy: 0.6051136255264282, Loss: 1.8211276531219482


100%|██████████| 22/22 [00:05<00:00,  4.27it/s]


...

Epochs: 58, Accuracy: 0.9914772510528564, Loss: 1.2864336967468262


100%|██████████| 22/22 [00:05<00:00,  4.31it/s]


Epochs: 59, Accuracy: 0.9943181872367859, Loss: 1.2838088274002075


100%|██████████| 22/22 [00:05<00:00,  4.30it/s]

Epochs: 60, Accuracy: 0.9943181872367859, Loss: 1.2843093872070312


## Evaluation

In [10]:
model.eval()

test_acc = 0.0
test_loss = 0.0
for audios, labels in tqdm(test):
    audios = audios.to(device)
    labels = labels.to(device)
    labels_one_hot = F.one_hot(labels,num_classes=8).type(torch.FloatTensor).to(device)

    pred = model(audios)
    loss = loss_function(pred,labels_one_hot)
    test_loss += loss
    test_acc += accuracy(pred, labels)

total_test_loss = test_loss / len(test)
total_test_acc = test_acc / len(test)
print(f"Accuracy: {total_test_acc}, Loss: {total_test_loss}")

100%|██████████| 6/6 [00:28<00:00,  4.71s/it]

Accuracy: 0.9635416865348816, Loss: 1.3298324346542358


In [ ]:
torch.save(model.state_dict(), "weights.pth")